In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from textblob import TextBlob
import collections
import statistics
import re
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.preprocessing import normalize
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
%matplotlib inline

In [2]:
train=pd.read_csv('new_set.csv')
train.head()

,Unnamed: 0,qid,question_text,target,period_count,word_length,char_length,polarity,subjectivity
0,0,00002165364db923c7e6,How did Quebec nationalists see their province...,0,0,13,72,0.0,0.0
1,1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0,0,16,81,0.0,0.0
2,2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0,0,10,67,0.0,0.0
3,3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0,0,9,57,0.0,0.0
4,4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0,0,15,77,0.0,0.0


In [3]:
def train_score(train1):
  train1.loc[:,'word'] = train1['question_text'].apply(lambda x: [re.sub('[^A-Za-z0-9]+', '', i) for i in x.split(" ")])
  bad_word_dict=collections.Counter([y for x in train1['word'][train1['target']==1] for y in x])
  good_word_dict=collections.Counter([y for x in train1['word'][train1['target']==0] for y in x])
  total_word_dict=collections.Counter([y for x in train1['word'] for y in x])
  for i in total_word_dict:
      if i not in bad_word_dict:
          bad_word_dict[i] = -1
      if i not in good_word_dict:    
          good_word_dict[i] = 2
  raw_word_score = {i: (bad_word_dict[i]*good_word_dict[i])/
                    (total_word_dict[i]) for i in total_word_dict}
  raw_word_score_2 = {i: (bad_word_dict[i]-good_word_dict[i])/
                    (total_word_dict[i]) for i in total_word_dict}
  mean_score = statistics.mean(list(raw_word_score.values()))
  stdev = statistics.stdev(list(raw_word_score.values()))

  mean_score_2 = statistics.mean(list(raw_word_score_2.values()))
  stdev_2 = statistics.stdev(list(raw_word_score_2.values()))
  standard_word_score = {i : (raw_word_score[i]-mean_score)/stdev for i in raw_word_score}
  standard_word_score_2 = {i : (raw_word_score_2[i]-mean_score_2)/stdev_2 for i in raw_word_score_2}
  train1.loc[:,'score_1'] = train1['word'].apply(lambda x: sum([raw_word_score[i] for i in x]))
  train1.loc[:,'score_2'] = train1['word'].apply(lambda x: sum([raw_word_score_2[i] for i in x]))
  return train1.drop(columns = ["question_text","word","target"]),raw_word_score,raw_word_score_2

In [18]:
cols=['question_text','target','period_count','char_length','polarity'] 
X=train[cols]
y=train['target']

X_train,y_train = X,y


X_train,score1,score2 = train_score(X_train)
X_train = ((X_train-X_train.min())/(X_train.max()-X_train.min()))

X_train,  y_train  = np.array(X_train), np.array(y_train)

In [30]:
logreg = LogisticRegression()
ranforest = RandomForestClassifier()
adaboo = AdaBoostClassifier()
GBC = GradientBoostingClassifier()

logreg.fit(X_train, y_train)
ranforest.fit(X_train, y_train)
adaboo.fit(X_train, y_train)
GBC.fit(X_train, y_train)

logreg_feature = logreg.predict(X_train)
ranforest_feature = ranforest.predict(X_train)
adaboo_feature = adaboo.predict(X_train)
GBC_feature = GBC.predict(X_train)

/Users/royluo/anaconda3/envs/devfest/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/royluo/anaconda3/envs/devfest/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [34]:
model = tf.keras.models.Sequential()
rate = .1
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(20, activation = tf.nn.relu))
#model.add(tf.keras.layers.Dense(32, activation = tf.nn.relu))
#model.add(tf.keras.layers.Dense(32, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(6, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(6, activation = tf.nn.relu))

model.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax))

model.compile(optimizer='adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])
model.fit(X_train,y_train,epochs = 2)
tf_feature = model.predict_classes([X_train])

Epoch 1/2
1306122/1306122 [==============================] - 86s 66us/step - loss: 0.1726 - acc: 0.9444
Epoch 2/2
1306122/1306122 [==============================] - 84s 64us/step - loss: 0.1564 - acc: 0.9468


In [35]:
gbm = xgb.XGBClassifier(objective= 'binary:logistic')

features_train = np.concatenate(([logreg_feature],[ranforest_feature],[adaboo_feature],
                           [GBC_feature],[tf_feature])).transpose()
gbm.fit(features_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

Now testing the final model with two real sentences. [1] means toxicity and insincerity, [0] means ok.

In [58]:
for i in range(2):
    question1 = input('Enter your text\n')
    #question1 = 'Has the U.S become the largest dictatorship in this world?'
    avg_1 = statistics.mean(list(score1.values()))
    avg_2 = statistics.mean(list(score2.values()))

    word_length = len(question1.split(' '))
    char_length = len(question1)
    period_count = question1.count('.')
    word = [re.sub('[^A-Za-z0-9]+', '', i) for i in question1.split(" ")]
    score_1 = sum([score1[i] if i in score1 else avg_1 for i in word ])
    score_2 = sum([score2[i] if i in score2 else avg_2 for i in word ])
    sent = TextBlob(question1).sentiment

    raw_input = [period_count,char_length,sent.polarity,score_1,score_2]
    new_set = train_score(X)[0]

    logreg_input = logreg.predict([question_input])
    ranforest_input = ranforest.predict([question_input])
    adaboo_input = adaboo.predict([question_input])
    GBC_input = GBC.predict([question_input])
    tf_input = model.predict_classes(np.array([question_input]))

    features = ['period_count','char_length','polarity','score_1','score_2']

    for i,feature in enumerate(features):
        M = new_set[feature].max()
        m = new_set[feature].min()
        raw_input[i] = (raw_input[i] - m) / (M-m)
    question_input = np.array(raw_input)
    print(gbm.predict(np.array([logreg_input,ranforest_input,adaboo_input,GBC_input,tf_input]).transpose()))

Enter your text
This tax increase is ridiculous, the republicans are despicable and should go to jail for this crime.


/Users/royluo/anaconda3/envs/devfest/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


[1]
Enter your text
Fuck the weather. I hate having to wear like a fat pig every time i go outside


/Users/royluo/anaconda3/envs/devfest/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


[1]
